##**Mounting Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importing Libraries**

In [2]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.0 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#Hyperparameter Optimization
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-3-ff41bca45932>:16: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


##**Determining File Path and Separating into Subfolders**

In [4]:
data_dir = '/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/'

data_path = "/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"
diabetic_path = os.path.join(data_path, "Diabetic Retinopathy")
normal_path = os.path.join(data_path, "Normal")

print(os.listdir("/content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/"))

['Glaucoma', 'Normal', 'Cataract', 'Diabetic Retinopathy']


##**Specifying Image Size**

In [5]:
img_height, img_width = 128, 128

##**Loading Images**

In [6]:
def load_images(folder_path, label):
    images = []
    labels = []
    for idx, filename in enumerate(os.listdir(folder_path)):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error: Unable to read image {img_path}")
            continue
        img = cv2.resize(img, (img_height, img_width))
        images.append(img)
        labels.append(label)
        print(f"Image {idx + 1} loaded from {img_path}")
    return np.array(images), np.array(labels)

diabetic_images, diabetic_labels = load_images(diabetic_path, 0)
normal_images, normal_labels = load_images(normal_path, 1)

Image 1 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10252_left.jpeg
Image 2 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10306_left.jpeg
Image 3 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10175_right.jpeg
Image 4 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10124_right.jpeg
Image 5 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10444_right.jpeg
Image 6 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10266_left.jpeg
Image 7 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10416_right.jpeg
Image 8 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retinopathy/10297_right.jpeg
Image 9 loaded from /content/drive/MyDrive/DerinOgrenme/RetinalImaging/Eyes/Diabetic Retino

##**Concatenating the Data Set as Cataract and Normal**

In [7]:
x = np.concatenate((diabetic_images, normal_images), axis=0)
y = np.concatenate((diabetic_labels, normal_labels), axis=0)

###Displaying the Number of Data Samples for X and Y

In [8]:
print("Number of data samples:", x.shape[0])
print("Number of data samples:", y.shape[0])

Number of data samples: 2172
Number of data samples: 2172


###Splitting the Data Set into Training (80%) and Test (20%) Sets

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [11]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [12]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [13]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.9942528605461121

Best val_accuracy So Far: 0.9942528605461121
Total elapsed time: 00h 06m 17s
Epoch 1/15
49/49 [==============================] - 4s 42ms/step - loss: 132.9240 - accuracy: 0.7876 - val_loss: 0.1448 - val_accuracy: 0.9483
Epoch 2/15
49/49 [==============================] - 2s 39ms/step - loss: 0.1757 - accuracy: 0.9603 - val_loss: 0.1309 - val_accuracy: 0.9770
Epoch 3/15
49/49 [==============================] - 2s 37ms/step - loss: 0.0767 - accuracy: 0.9738 - val_loss: 0.0793 - val_accuracy: 0.9885
Epoch 4/15
49/49 [==============================] - 2s 36ms/step - loss: 0.0661 - accuracy: 0.9808 - val_loss: 0.0527 - val_accuracy: 1.0000
Epoch 5/15
49/49 [==============================] - 2s 37ms/step - loss: 0.0588 - accuracy: 0.9859 - val_loss: 0.1212 - val_accuracy: 1.0000
Epoch 6/15
49/49 [==============================] - 2s 39ms/step - loss: 0.0394 - accuracy: 0.9872 - val_loss: 0.0987 - val_accuracy: 0.9943
Epoch 7/15

In [14]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

6/6 [==============================] - 0s 10ms/step - loss: 0.0973 - accuracy: 0.9655
Validation Accuracy:  0.9655172228813171
14/14 [==============================] - 0s 29ms/step - loss: 0.0894 - accuracy: 0.9678
Test Accuracy:  0.9678161144256592
49/49 [==============================] - 0s 8ms/step - loss: 0.0867 - accuracy: 0.9706
Training Accuracy:  0.9705694317817688


In [15]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

14/14 [==============================] - 0s 5ms/step
Test Accuracy: 0.9678
Precision: 0.9692
Recall: 0.9678
F1 Score: 0.9678
Confusion Matrix:
[[198  13]
 [  1 223]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.97       211
           1       0.94      1.00      0.97       224

    accuracy                           0.97       435
   macro avg       0.97      0.97      0.97       435
weighted avg       0.97      0.97      0.97       435



###Splitting the Data Set into Training (65%) and Test (35%) Sets

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 38)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.10, random_state = 38)

###Training and Finalizing the CNN Model

In [17]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [18]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [19]:
def build_model(hp):
    model = Sequential()

    # Convolutional Layer 1
    model.add(Conv2D(filters=hp.Int('conv1_filters', min_value=32, max_value=128, step=16),
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(img_height, img_width, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv1', min_value=0.2, max_value=0.5, step=0.1)))

    # Convolutional Layer 2
    model.add(Conv2D(filters=hp.Int('conv2_filters', min_value=64, max_value=256, step=16),
                     kernel_size=(3, 3),
                     padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=hp.Float('dropout_conv2', min_value=0.2, max_value=0.5, step=0.1)))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer 1
    model.add(Dense(units=hp.Int('dense1_units', min_value=256, max_value=1024, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout1', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 2
    model.add(Dense(units=hp.Int('dense2_units', min_value=128, max_value=512, step=32)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0.2, max_value=0.5, step=0.1)))

    # Dense Layer 3
    model.add(Dense(units=hp.Int('dense3_units', min_value=64, max_value=256, step=16)))
    model.add(Activation('relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0.2, max_value=0.5, step=0.1)))

    # Output Layer
    model.add(Dense(2, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [20]:
# Creating the tuner using RandomSearch
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # Maximum number of models to try
    directory='my_dir',  # Directory to store saved models and results
    project_name='retinal_imaging_hyperparameter_tuning'  # Name of the tuning project
)

# Fitting the tuner with training data
tuner.search(x_train, train_yCl, epochs=10, validation_data=(x_val, valid_yCl))

# Getting the best hyperparameters and building the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# Training the best model on the training set
best_model.fit(x_train, train_yCl, epochs=15, validation_data=(x_val, valid_yCl))

Reloading Tuner from my_dir/retinal_imaging_hyperparameter_tuning/tuner0.json
Epoch 1/15
40/40 [==============================] - 4s 56ms/step - loss: 120.8122 - accuracy: 0.6667 - val_loss: 0.3684 - val_accuracy: 0.9366
Epoch 2/15
40/40 [==============================] - 1s 37ms/step - loss: 0.1982 - accuracy: 0.9354 - val_loss: 0.2124 - val_accuracy: 0.9930
Epoch 3/15
40/40 [==============================] - 1s 36ms/step - loss: 0.0781 - accuracy: 0.9740 - val_loss: 0.0841 - val_accuracy: 0.9930
Epoch 4/15
40/40 [==============================] - 1s 37ms/step - loss: 0.0527 - accuracy: 0.9819 - val_loss: 0.0895 - val_accuracy: 0.9930
Epoch 5/15
40/40 [==============================] - 2s 38ms/step - loss: 0.0360 - accuracy: 0.9874 - val_loss: 0.0538 - val_accuracy: 1.0000
Epoch 6/15
40/40 [==============================] - 2s 39ms/step - loss: 0.0340 - accuracy: 0.9882 - val_loss: 0.0404 - val_accuracy: 1.0000
Epoch 7/15
40/40 [==============================] - 2s 38ms/step - loss: 0

In [21]:
score_valid = best_model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = best_model.evaluate(x_test, test_yCl)
print("Test Accuracy: ", score_test[1])

score_train = best_model.evaluate(x_train, train_yCl)
print("Training Accuracy: ", score_train[1])

5/5 [==============================] - 0s 9ms/step - loss: 0.0609 - accuracy: 0.9859
Validation Accuracy:  0.98591548204422
24/24 [==============================] - 0s 18ms/step - loss: 0.0651 - accuracy: 0.9921
Test Accuracy:  0.9921156167984009
40/40 [==============================] - 0s 6ms/step - loss: 0.0494 - accuracy: 0.9898
Training Accuracy:  0.9897556900978088


In [22]:
# Making predictions on the test data
y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_yCl, axis=1)

# Calculating metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes, labels=np.unique(y_true))

# Printing the results
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Displaying the Confusion Matrix
print('Confusion Matrix:')
print(conf_matrix)

# Showing the Classification Report
print('Classification Report:')
print(class_report)

24/24 [==============================] - 0s 4ms/step
Test Accuracy: 0.9921
Precision: 0.9922
Recall: 0.9921
F1 Score: 0.9921
Confusion Matrix:
[[365   6]
 [  0 390]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       371
           1       0.98      1.00      0.99       390

    accuracy                           0.99       761
   macro avg       0.99      0.99      0.99       761
weighted avg       0.99      0.99      0.99       761

